In [1]:
#feature extraction from previous segmented images 
#to extract features for each image in the dataset such as texture, color, shape, size, etc
#and store them in a file
import cv2
import os
import numpy as np
from skimage.feature import hog
from skimage import data, exposure

In [ ]:
class feature_extract:
    def feature_extract(self, image):
        pass

    def feature_extract_all(self, images):
        pass

    
    def feature_extract_all_batch(self, images):
        pass

    